In [1]:
#Import packages.
import pandas as pd
import json

In [21]:
def extract_ann(filename, column_name, category_num):
    if ((not isinstance(category_num, list)) | (not isinstance(column_name, list))):
        print("Your category is not a list")
    else:
        #Open the faces json.
        with open(filename) as json_data:
            data = json.load(json_data)
        df = pd.DataFrame(data['images'])
        ann = pd.DataFrame(data['annotations'])
    
        for category, name in zip(category_num, column_name):
            data_sub = ann[ann['category_id']==category]
            
            data_counts = data_sub.groupby(['image_id']).count()
            data_counts_cat = data_counts[['category_id']]
            count_name = f"{name}_count"
            data_counts_cat = data_counts_cat.rename(columns={"category_id": count_name})
            
            sub_set = set(data_sub['image_id'].to_list())
            df[name] = df['id'].isin(sub_set).astype(int)
            df = df.merge(data_counts_cat, how='outer', left_on='id', right_index=True)
    return df
    

In [22]:
data1 = extract_ann("./cocoface_instances_train2017.json", ['people', 'faces'], [1,2])
data2 = extract_ann("./instances_train2017.json", ['people_coco'], [1])
data3 = extract_ann("./cocoface_instances_val2017.json", ['people','faces'], [1,2])
data4 = extract_ann("./instances_val2017.json", ['people_coco'], [1])

In [26]:
data_faces = pd.concat([data1, data3])
data_coco = pd.concat([data2, data4])
data_all = data_coco.merge(data_faces, how='outer', on=['id', 'file_name', 'coco_url'])
data_all = data_all[['id', 'file_name', 'coco_url', 'people_coco', 'people', 'faces','people_coco_count', 'people_count','faces_count']]

#Replace nans with 0s.
data_all.fillna(0, inplace=True)

In [28]:
#Save to csv.
data_all.to_csv("./coco_annotations_socnosoc.csv")